In [ ]:
%pip install certifi
%pip install pyarrow

In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import certifi

In [21]:
header_data=pd.read_parquet('header_data.parquet',"pyarrow")
urls=pd.read_parquet('urls.parquet',"pyarrow")

In [3]:
#header_data.head()
urls.head(10)

,url
0,almaleasdancestudio.com
1,reddeerspecialties.com
2,nopopaws.com
3,burlingamedentalarts.com
4,sachsfamilydental.com
5,bhgidoc.com
6,billcollectorcharter.com
7,imperialtrainingcenter.com
8,physiocan.com
9,1stle.com


In [22]:
urls['company_name']=urls.url.str.replace(".com",'')
urls['company_name']

C:\Users\Pranav Belmannu\AppData\Local\Temp\ipykernel_39480\3084780711.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  urls['company_name']=urls.url.str.replace(".com",'')


0         almaleasdancestudio
1          reddeerspecialties
2                    nopopaws
3        burlingamedentalarts
4           sachsfamilydental
                 ...         
25782                hidytidy
25783               rjtherapy
25784              peaseranch
25785       oakcreekranch.net
25786     valleywaterdistrict
Name: company_name, Length: 25787, dtype: object

In [23]:
urls['company_name']=urls.company_name.str.replace(".net",'')
urls['company_name']

C:\Users\Pranav Belmannu\AppData\Local\Temp\ipykernel_39480\2285090443.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  urls['company_name']=urls.company_name.str.replace(".net",'')


0         almaleasdancestudio
1          reddeerspecialties
2                    nopopaws
3        burlingamedentalarts
4           sachsfamilydental
                 ...         
25782                hidytidy
25783               rjtherapy
25784              peaseranch
25785           oakcreekranch
25786     valleywaterdistrict
Name: company_name, Length: 25787, dtype: object

In [37]:
urls['company_name']=urls.company_name.str.replace(".org",'')
urls['company_name']

C:\Users\Pranav Belmannu\AppData\Local\Temp\ipykernel_39480\2140758328.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  urls['company_name']=urls.company_name.str.replace(".org",'')


0         almaleasdancestudio
1          reddeerspecialties
2                    nopopaws
3        burlingamedentalarts
4           sachsfamilydental
                 ...         
25782                hidytidy
25783               rjtherapy
25784              peaseranch
25785           oakcreekranch
25786     valleywaterdistrict
Name: company_name, Length: 25787, dtype: object

In [11]:
import nltk
from nltk.corpus import words
from nltk.tokenize import word_tokenize
from nltk.data import find

In [12]:
try:
    find('corpora/words.zip')
except:
    nltk.download('words')

[nltk_data] Downloading package words to C:\Users\Pranav
[nltk_data]     Belmannu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


In [25]:
word_list = set(words.words())

In [ ]:
def extract_company_name(soup):
    
    if soup.find(id,'SITE_FOOTER'):
        text = soup.find(id,'SITE_FOOTER').get_text(strip=True)
        return text.split('\n')[0]
    
    for potential_name in ['company', 'business', 'organization', 'name']:
        meta = soup.find('meta', attrs={'name': potential_name})
        if meta and meta.get('content'):
            return meta.get('content')
    
    return None

In [27]:
def clean_company_name(name):
    # Replace underscores or excess spaces if necessary
    return ' '.join(name.split())

In [26]:
def break_into_words(text, word_list):
    start = 0
    result = []
    for i in range(1, len(text) + 1):
        if text[start:i] in word_list:
            result.append(text[start:i])
            start = i
    return ' '.join(result)

In [17]:
urls['company_name'] = urls['url'].apply(lambda x: break_into_words(x, word_list))

In [28]:
urls['company_name'] = urls['company_name'].apply(clean_company_name)

In [29]:
urls['company_name']

0         almaleasdancestudio
1          reddeerspecialties
2                    nopopaws
3        burlingamedentalarts
4           sachsfamilydental
                 ...         
25782                hidytidy
25783               rjtherapy
25784              peaseranch
25785           oakcreekranch
25786     valleywaterdistrict
Name: company_name, Length: 25787, dtype: object

In [ ]:
https='https://'
http='http://'

companies = []  # List to store BeautifulSoup objects
company_names = []  # List to store extracted company names
urls['request_failed'] = False  # Initialize a column to mark failures

for index, row in urls.iterrows():
    url = row['url']
    full_url = https + url if not url.startswith('http') else url

    try:
        response = requests.get(full_url, allow_redirects=True, timeout=5)
        response.raise_for_status()  # Raise an exception for HTTP errors
        print(f"Success with HTTPS: {full_url}")
        soup = BeautifulSoup(response.content, 'html.parser')
        companies.append(soup)  # Store the BeautifulSoup object

        # Extract and store the company name
        company_name = extract_company_name(soup)
        company_names.append(company_name)
        
    except requests.exceptions.RequestException as e:
        print(f"HTTPS failed for {full_url}: {e}")

        full_url = http + url if not url.startswith('https') else url
        try:
            response = requests.get(full_url, allow_redirects=True, timeout=5)
            response.raise_for_status()  # Raise an exception for HTTP errors
            print(f"Success with HTTP: {full_url}")
            soup = BeautifulSoup(response.content, 'html.parser')
            companies.append(soup)  # Store the BeautifulSoup object

            # Extract and store the company name
            company_name = extract_company_name(soup)
            company_names.append(company_name)
            
        except requests.exceptions.RequestException as e:
            print(f"HTTP also failed for {full_url}: {e}")
            urls.at[index, 'request_failed'] = True  # Mark the failure in the DataFrame
            continue  # Skip this URL if both HTTPS and HTTP fail
    else:
        urls.at[index, 'request_failed'] = False  # Mark success

# After the loop, 'companies' will contain the BeautifulSoup objects for each accessible URL
# The 'company_names' list will contain extracted company names
# The 'urls' DataFrame will have a 'request_failed' column indicating failure

In [24]:
header_data.business_name

0                           Lift and Level
1           Way of Faith Christian Academy
2        Urology Center of Central Florida
3                 Housing Trust of America
4             Theobald Family Chiropractic
                       ...                
25782     Missouri Rural Water Association
25783            Consumer Choice Marketing
25784                 Tennessee Electrical
25785      Horsham Water & Sewer Authority
25786                     Ohio Gas Company
Name: business_name, Length: 25787, dtype: object

In [32]:
header_data.head()
header_data['url']=header_data.business_name.str.replace(" ","")
header_data['url']=header_data['url'].str.lower()
header_data['url']=header_data['url']+'.com'
header_data['url']



0                         liftandlevel.com
1           wayoffaithchristianacademy.com
2        urologycenterofcentralflorida.com
3                housingtrustofamerica.com
4           theobaldfamilychiropractic.com
                       ...                
25782    missouriruralwaterassociation.com
25783          consumerchoicemarketing.com
25784              tennesseeelectrical.com
25785      horshamwater&sewerauthority.com
25786                   ohiogascompany.com
Name: url, Length: 25787, dtype: object

In [38]:
naics_dict = dict(zip(header_data['url'],header_data['NAICS2']))
urls['NAICS2'] = urls['url'].apply(lambda x: naics_dict.get(x, 'Not Found'))
urls.head(24)

,url,company_name,NAICS2
0,almaleasdancestudio.com,almaleasdancestudio,Not Found
1,reddeerspecialties.com,reddeerspecialties,22
2,nopopaws.com,nopopaws,11
3,burlingamedentalarts.com,burlingamedentalarts,62
4,sachsfamilydental.com,sachsfamilydental,62
5,bhgidoc.com,bhgidoc,Not Found
6,billcollectorcharter.com,billcollectorcharter,Not Found
7,imperialtrainingcenter.com,imperialtrainingcenter,61
8,physiocan.com,physiocan,62
9,1stle.com,1stle,Not Found
